In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string
import math
import heapq


In [272]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []

In [287]:


#######################################importing
def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

#######################################Normalization
def delete_punctuations(doc):
    punctuations = '؟،:؛!»«()[]*,{.}@!?،؛<>#$&!~"\|-_+'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = edited_doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    for i in inverted_indexes:
        if len(i[1])/len(df) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens): 
    for i in range(0, len(tokens)):
        # print(verb_roots)
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        # print("common")
                        break
                if common == False:
                    my_regex = r"\b^"+root[1]+r"\.*"
                    x = re.findall(my_regex, tokens[i])
 
                    if len(x) == 0:  
                        my_regex = r"\b...*"+root[1]+r"\b"
                        x1 = re.findall(my_regex, s[0])
                        if len(x1)==0:         
                            # print("root")
                            tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

#######################################tokenization
def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        

        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            if len(token) > 0:
                temp = []
                temp.append(token)
                temp.append(df.id[i])
                tokens.append(temp)
    tokens.sort()
    return tokens

#######################################create inverted index
def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)

            (unique, counts) = np.unique(doc_temp, return_counts=True)
            # frequencies = np.array((unique, counts), dtype='i4,f4').T.view(np.recarray)       
            frequencies = []      
            for i in range(len(unique)):
                temp1 = []
                temp1.append(unique[i])
                temp1.append(counts[i])
                frequencies.append(temp1)
            temp.append(frequencies)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

#######################################calculate weights
def calculate_tfidf(df, inverted_indexes):
    doc_length = np.zeros(len(df)+1)
    len_df = len(df)
    for i in range(len(inverted_indexes)):
        if len(inverted_indexes[i][1]) > 0:
            # print(weighted_inverted_index[i])
            # print(len(inverted_indexes[i][1]))
            idf = math.log(len_df/len(inverted_indexes[i][1]), 10)
            inverted_indexes[i].append(idf)
            # print("idf = ", idf)
            for j in range(len(inverted_indexes[i][1])):
                tf = 1 + math.log(inverted_indexes[i][1][j][1], 10)
                # print("tf = ", tf)
                w = tf*idf
                # print("w : ",w)
                inverted_indexes[i][1][j][1] = w
                doc_length[inverted_indexes[i][1][j][0]] = doc_length[inverted_indexes[i][1][j][0]] + np.power(w, 2)
                
    doc_length =  np.sqrt(doc_length)
    return inverted_indexes, doc_length

#######################################create champion list
def create_championList(inverted_indexes, r):
    limit = r
    championList = []
    for i in range(len(inverted_indexes)):
        temp = []
        temp.append(inverted_indexes[i][0])
        temp_postingList = list(inverted_indexes[i][1])
        temp_postingList.sort(key=lambda x: x[1], reverse=True)

        temp1 = []
        r = limit
        if r > len(temp_postingList):
            r = len(temp_postingList)
        for j in range(r):
            # print(temp_postingList[j])
            temp1.append(temp_postingList[j])
        temp.append(temp1)
        if len(inverted_indexes[i]) == 3:
            temp.append(inverted_indexes[i][2])
        championList.append(temp)
    return championList

#######################################Searching
def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            print("token*****: ", token[0])
            return token[1], token[2]
    
    print("this token not exist in our database")
    return -1, -1

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    print(our_res)
    

def binary_search(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        docs, idf = search_token(inverted_indexes, tokenized_query[i])
        if docs != -1:
            temp_docs_id = pd.DataFrame(docs)
            docs_id = pd.DataFrame(docs_id.append(temp_docs_id))

    if len(docs_id) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    print_res(df, docs_id)


def ranked_search(df, inverted_indexes, query, doc_length, k, using_heap):
    res = []
    docs_id = pd.DataFrame()
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)

    doc_score_temp = []
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        # print(docs)


        if docs != -1:
            # print(docs)
            # print(idf)
            w_termInQuery = tf_query*idf
            # print("weight term in q: ", tf_query, idf, w_termInQuery)

            for i in range(len(docs)):
                temp = []
                temp.append(docs[i][0])
                # print("docs[i][1]: ",docs[i][1])
                temp.append(w_termInQuery*docs[i][1])
                doc_score_temp.append(temp)

    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1
    doc_score_temp.sort()
    doc_score = []
    doc_score.append(doc_score_temp[0])
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
        else:
            doc_score.append(doc_score_temp[i])
    
    for i in range(len(doc_score)):
        doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked

    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    print_res(df, docs_id)


In [274]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [4]:
tokens = tokenize(df)

In [5]:
inverted_indexes = create_inverted_indexes(tokens)

In [6]:
inverted_indexes, doc_length = calculate_tfidf(df, inverted_indexes)

In [7]:
championList = create_championList(inverted_indexes, 4)

In [16]:
query = "تیم سپاهان فرصت"
# query_processing(df, inverted_indexes, query)

doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = True)
# doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = False)

doc_score = ranked_search(df, inverted_indexes, query, doc_length, k = 10, using_heap = True)

binary_search(df, inverted_indexes, query)

token*****:  تیم
token*****:  سپاهان
token*****:  فرصت
     id      rank                                                url
5  1269  0.336423  https://www.isna.ir/news/98051508035/برنامه-کا...
1   250  0.300315  https://www.isna.ir/news/99041007307/سپاهان-نی...
2   307  0.196843  https://www.isna.ir/news/99042820646/سپاهان-مق...
3   373  0.186529  https://www.isna.ir/news/99052216370/درد-فوتبا...
0    58  0.120334  https://www.isna.ir/news/99012514072/ورزشکاران...
7  3539  0.097260  https://www.isna.ir/news/99041611905/فرصتی-برا...
9  5599  0.095828  https://www.isna.ir/news/99022719009/دردهای-شک...
8  4391  0.089950  https://www.isna.ir/news/98020703682/ماشین-ساز...
4   663  0.036668  https://www.isna.ir/news/99081610283/ماندگاری-...
6  1545  0.029156  https://www.isna.ir/news/98101108272/انتقاد-به...
token*****:  تیم
token*****:  سپاهان
token*****:  فرصت
     id      rank                                                url
0   181  0.476196  https://www.isna.ir/news/99032012883/سومین-

In [9]:
print(inverted_indexes[1][1])
temp_postingList = list(inverted_indexes[1][1])
temp_postingList.sort(key=lambda x: x[1], reverse=True)
print(temp_postingList)
print(inverted_indexes[1][1])


[[6768, 3.3679767852945943], [6774, 4.381838822368215], [6780, 4.381838822368215]]
[[6774, 4.381838822368215], [6780, 4.381838822368215], [6768, 3.3679767852945943]]
[[6768, 3.3679767852945943], [6774, 4.381838822368215], [6780, 4.381838822368215]]


In [10]:
# print(championList[5000])
# print(inverted_indexes[5000])
print(len(tokens))

1700417


In [11]:
s = "گفت‌و‌گو"
x = s.split()
print(x)

['گفت\u200cو\u200cگو']


In [12]:
print(search('شنید', 'شنید'))

<re.Match object; span=(0, 4), match='شنید'>


In [28]:
%%time
query = "بازی پرسپولیس و استقلال چند شنبه است؟"

doc_score = ranked_search(df, inverted_indexes, query, doc_length, k = 20, using_heap = False)


this token not exist in our database
token*****:  استقلال
token*****:  بازی
token*****:  شنبه
token*****:  پرسپولیس
token*****:  چند
      id      rank                                                url
17  1480  0.634205  https://www.isna.ir/news/98090805350/حضور-فتحی...
19  1613  0.621676  https://www.isna.ir/news/98111511080/فروش-۱۷-ه...
2    213  0.601773  https://www.isna.ir/news/99033018956/موافقت-او...
12  1042  0.595021  https://www.isna.ir/news/98010902726/شوک-بزرگ-...
9    701  0.577228  https://www.isna.ir/news/99082820069/بازیکن-مد...
14  1051  0.568810  https://www.isna.ir/news/98011705553/آمادگی-صی...
1     75  0.566203  https://www.isna.ir/news/99020805715/با-دستور-...
13  1043  0.554887  https://www.isna.ir/news/98010902616/گزارشگر-د...
11   915  0.545152  https://www.isna.ir/news/99111309632/شکایت-است...
18  1566  0.532947  https://www.isna.ir/news/98101713761/افشین-پیر...
10   708  0.525139  https://www.isna.ir/news/99083020917/مهدی-خانی...
4    287  0.514573  https:/

In [29]:
%%time
query = "بازی پرسپولیس و استقلال چند شنبه است؟"

doc_score = ranked_search(df, championList, query, doc_length, k = 20, using_heap = True)


this token not exist in our database
token*****:  استقلال
token*****:  بازی
token*****:  شنبه
token*****:  پرسپولیس
token*****:  چند
      id      rank                                                url
9   1269  0.451469  https://www.isna.ir/news/98051508035/برنامه-کا...
6    582  0.252221  https://www.isna.ir/news/99072418213/در-آخرین-...
5    579  0.230285  https://www.isna.ir/news/99072418443/بشار-رسن-...
15  4068  0.204922  https://www.isna.ir/news/99100403028/بررسی-کلی...
7    738  0.194617  https://www.isna.ir/news/99091310473/بازگشایی-...
4    309  0.165733  https://www.isna.ir/news/99042921462/فنونی-زاد...
2    278  0.161771  https://www.isna.ir/news/99041914462/پشت-پرده-...
1     81  0.111597  https://www.isna.ir/news/99021309405/معاون-ساز...
10  1404  0.098346  https://www.isna.ir/news/98080201236/به-مجموعه...
12  1660  0.095537  https://www.isna.ir/news/98120706128/تکسیرا-در...
8    752  0.085558  https://www.isna.ir/news/99091713534/صعود-استق...
3    307  0.068299  https:/

In [291]:
s = ['می‌روم']

print(replaceWithRoot(s))

['رفتن']
